### Import api_crawler

[Code for api_crawler](https://github.com/biothings/JSON-LD_BioThings_API_DEMO/blob/master/src/api_crawler.py)

In [1]:
from api_crawler import uri_query

### Given a variant hgvs id, looking for ncbi gene id related to it

In [2]:
uri_query(input_value='chr12:g.103234255C>T', input_type='http://identifiers.org/hgvs/', output_type='http://identifiers.org/ncbigene/')

1 api(s) could be utilized based on the input_type, output_type you provided: ['myvariant.info']
myvariant.info returns the following results based on your query: 5053


[{'myvariant.info': '5053'}]

### From the ncbi gene we found in previous query, get all wikipathways ids related

In [3]:
uri_query(input_value='5053', input_type='http://identifiers.org/ncbigene/', output_type='http://identifiers.org/wikipathways/')

1 api(s) could be utilized based on the input_type, output_type you provided: ['mygene.info']
mygene.info returns the following results based on your query: ['WP706', 'WP550']


[{'mygene.info': ['WP706', 'WP550']}]

### Breakdown of uri_query function

The following code shows each step involved in uri_query function demonstrated above. 

[Metadata information about BioThings API(config)](https://github.com/biothings/JSON-LD_BioThings_API_DEMO/blob/master/src/config.py)
[code for biothings_helper](https://github.com/biothings/JSON-LD_BioThings_API_DEMO/blob/master/src/biothings_helper.py)

#### Step 1: Specify input and output

In [4]:
input_value = '5053'
input_type='http://identifiers.org/ncbigene/'
output_type='http://identifiers.org/wikipathways/'

#### Step 2: Iterate through API metadata info, and find corresponding API based on input & output

In [5]:
from config import AVAILABLE_API_SOURCES
from biothings_helper import find_id_from_uri, find_value_from_output_type, query_ids_from_output_type
from api_crawler import api_lookup

In [6]:
# convert to internal input name and output name
input_name = find_id_from_uri(input_type)
output_name = find_id_from_uri(output_type)
# look up api in api metadata info
api_results = api_lookup(input_name, output_name)
print(api_results)

[{'type': 'annotate', 'api': 'mygene.info'}]


#### Step 3: Make API call

In [7]:
# construct url based on metadata info
url = AVAILABLE_API_SOURCES[api_results[0]['api']]['annotate_syntax'].replace('{{input}}', input_value)
# make API call
import requests
doc = requests.get(url).json()
#for better display in ipython notebook, we are not printing the whole json doc here
#the following code could be used to display the json_doc returned from api call
# print(doc)

#### Step 4: Transform JSON doc to JSON-LD doc and Nquads format

In [9]:
# load context file
context = requests.get(AVAILABLE_API_SOURCES[api_results[0]['api']]["jsonld"]["context_file_path"]).json()
# construct json-ld doc
doc.update(context)
# transform json-ld doc to nquads format
from pyld import jsonld
t = jsonld.JsonLdProcessor()
nquads = t.parse_nquads(jsonld.to_rdf(doc, {'format': 'application/nquads'}))['@default']
print(nquads[1])
# for better display in ipython notebook, we are not printing the whole nquads doc here\
# the following code could be used to display the whole nquads doc
# print(nquads)

{'predicate': {'type': 'IRI', 'value': 'http://identifiers.org/ensembl.protein/'}, 'object': {'type': 'literal', 'value': 'ENSP00000303500', 'datatype': 'http://www.w3.org/2001/XMLSchema#string'}, 'subject': {'type': 'blank node', 'value': '_:b0'}}


#### Step 5: Fetch value using URI from Nquads format

In [10]:
value_list = []
for item in nquads:
    if item['predicate']['value'] == output_type:
        value_list.append(item['object']['value'])
value = list(set(value_list))
print(value)

['WP706', 'WP550']
